# Imports etc.

In [1]:
import ipdb
import numpy as np
import os
import pandas as pd
import plotnine as gg
gg.theme_set(gg.theme_classic)

from Functions import get_info_from_fullID, add_meta_column, sigmoid, get_session #, load_mouse_data, load_one_measure, remove_na_trials, 

In [2]:
# fitted_param_dir = 'C:/Users/maria/MEGAsync/SLCN/PShumanData/fitting/mice/'
mouse_data_dir = 'C:/Users/maria/MEGAsync/SLCN/PSMouseData'
# plot_dir = 'C:/Users/maria/MEGAsync/SLCN/models/plots'
# if not os.path.exists(plot_dir):
#     os.mkdir(plot_dir)

In [3]:
n_trial_cutoff = 500  # 500 as of 2020/07/21
n_session_cutoff = 10  # 10 as of 2020/07/21
rt_cut_off = 4  # 4 seconds as of 2020/08/04; 10 seconds as of 2020/07/29

# Functions to load and format mouse data

In [4]:
def load_one_measure(name, data_dir):
    
    measure_j = pd.read_csv(os.path.join(data_dir, 'Juvi_{}.csv'.format(name))).T.values  # after reading in: [trials x animals]
    measure_a = pd.read_csv(os.path.join(data_dir, 'Adult_{}.csv'.format(name))).T.values
    measure_dat = np.hstack([measure_j, measure_a])
    
    return pd.DataFrame(measure_dat)

# # Example use
# load_one_measure('Reward', mouse_data_dir)

In [5]:
def remove_na_trials(measure_dat, missed_trials):
    
    measure_dat[missed_trials] = np.nan
    measure_dat = measure_dat.apply(lambda x: pd.Series(x.dropna().values))
    
    return measure_dat

# # Example use
# remove_na_trials(load_one_measure('Reward', mouse_data_dir))

In [6]:
# # rts = load_one_measure('ITI', mouse_data_dir)  # Lung-Hao: ITI is the time of last nose poke event (in or out) of previous trial to center poke of current trial. So it's the ITI proceeding the current trial. The first trial has ITI because we removed the trials before first switch.
# # actions = load_one_measure('Choice', mouse_data_dir)
# # rts[rts > rt_cut_off] = np.nan
# print(np.mean((rts > rt_cut_off).stack()))
# print(np.mean((np.isnan(actions).stack())))
# print(np.mean((rts > rt_cut_off).stack()) + np.mean((np.isnan(actions).stack())))
# np.mean(((rts > rt_cut_off) | (np.isnan(actions))).stack())

In [7]:
def load_mouse_data(data_dir):

    # Load mouse data
    rewards = load_one_measure('Reward', mouse_data_dir)
    rts = load_one_measure('ITI', mouse_data_dir)  # Lung-Hao: ITI is the time of last nose poke event (in or out) of previous trial to center poke of current trial. So it's the ITI proceeding the current trial. The first trial has ITI because we removed the trials before first switch.
    actions = load_one_measure('Choice', mouse_data_dir)
    correct_actions = load_one_measure('TaskData', mouse_data_dir)
    corrects = (actions == correct_actions).astype('int')  # When did mice choose the right action?

    fullID_j = pd.read_csv(os.path.join(data_dir, 'Juvi_AnimalID.csv')).T.values.flatten()
    fullID_a = pd.read_csv(os.path.join(data_dir, 'Adult_AnimalID.csv')).T.values.flatten()
    fullIDs = np.concatenate([fullID_j, fullID_a])
    
    # Remove na trials and too-fast trials by shifting up later trials
    missed_trials = np.isnan(actions) | (rts > rt_cut_off)
    actions = remove_na_trials(actions, missed_trials)
    rewards = remove_na_trials(rewards, missed_trials)
    rts = remove_na_trials(rts, missed_trials)
    corrects = remove_na_trials(corrects, missed_trials)
    correct_actions = remove_na_trials(correct_actions, missed_trials)

    # Make sure all dataframes have the same shape
    assert np.shape(rewards) == np.shape(actions)
    assert np.shape(corrects) == np.shape(correct_actions)
    assert np.shape(rewards) == np.shape(correct_actions)

    return {
        'actions': actions,
        'rewards': rewards,
        'corrects': corrects,
        'rts': rts,
        'correct_actions': correct_actions,
        'fullIDs': fullIDs,
    }

# # Example use
# raw_dat = load_mouse_data(mouse_data_dir)
# raw_dat

In [8]:
def get_subj_dat(raw_dat, fullID, i):
    
    subj_dat = {
        'action': raw_dat['actions'].loc[:, i],
        'reward': raw_dat['rewards'].loc[:, i],
        'rt': raw_dat['rts'].loc[:, i],
        'correct': raw_dat['corrects'].loc[:, i],
        'trial': range(len(raw_dat['actions'].loc[:, i])),
        'block': np.append([0], np.cumsum(np.abs(np.diff(raw_dat['correct_actions'].loc[:, i])))),
        'fullID': [fullID] * len(raw_dat['actions'].loc[:, i]),
        }

    # Make sure we're not introducing nans
    assert np.mean(np.isnan(raw_dat['rts'].loc[:, i])) == np.mean(np.isnan(subj_dat['rt']))

    return pd.DataFrame(subj_dat)

# # Example use
# get_subj_dat(raw_dat, 0, 0)

In [9]:
def get_ago_col(subj_dat, colname, n):
    
    return np.append(n * [np.nan], subj_dat[colname][:-n])

# # Example use
# subj_dat = get_subj_dat(raw_dat, raw_dat['fullIDs'][0], 0)
# colname = 'action'
# n = 2

# subj_dat['{}_{}ago'.format(colname[:3], n)] = get_ago_col(subj_dat, colname, n)
# subj_dat

In [10]:
def format_mouse_data(raw_dat, trials2switch_lower=3):

    true_dat = pd.DataFrame()
    for i, fullID in enumerate(raw_dat['fullIDs']):

        print("Formatting mouse-session {} of {}.".format(i, len(raw_dat['fullIDs'])), end='\r')

        subj_dat = get_subj_dat(raw_dat, fullID, i)
        
        # Add fullID info
        for col in ['agegroup', 'sex', 'age', 'animal']:
            subj_dat[col] = get_info_from_fullID(fullID, col)

        # Add session info (within animal)
        for age in np.unique(subj_dat.age):
            
            # Get ago columns (rew_1ago, act_1ago, rew_2ago, ...)
            for colname in ['reward', 'correct', 'action']:
                for n in range(1, 20):
                    subj_dat['{}_{}ago'.format(colname[:3], n)] = get_ago_col(subj_dat, colname, n)

            # Add block info (within session, within animal)
            for block in np.unique(subj_dat.block):
                sub_idx = (subj_dat.block == block) & (subj_dat.age == age)
                block_length = sum(sub_idx)

                if block_length > trials2switch_lower + 2:
                    block_trial = range(block_length)
                    trials2switch = np.append(np.arange(block_length-trials2switch_lower),
                                              np.arange(-trials2switch_lower, 0))
                else:
                    block_trial = [np.nan] * block_length
                    trials2switch = [np.nan] * block_length

                subj_dat.loc[sub_idx, 'block_length'] = block_length
                subj_dat.loc[sub_idx, 'block_trial'] = block_trial
                subj_dat.loc[sub_idx, 'trials2switch'] = trials2switch
        
        true_dat = true_dat.append(subj_dat)
        
    # Add columns
    for animal in np.unique(true_dat.animal):
        true_dat.loc[true_dat.animal == animal, 'session'] = get_session(true_dat.loc[true_dat.animal == animal])
    true_dat['rew_1&2ago'] = [str(r1)[0] + str(r2)[0] for r1, r2 in zip(true_dat['rew_1ago'], true_dat['rew_2ago'])]  ###
    true_dat['cor_1&2ago'] = [str(r1)[0] + str(r2)[0] for r1, r2 in zip(true_dat['cor_1ago'], true_dat['cor_2ago'])]  ###
    true_dat['act_1&2ago'] = [str(r1)[0] + str(r2)[0] for r1, r2 in zip(true_dat['act_1ago'], true_dat['act_2ago'])]  ###
    true_dat['meta'] = add_meta_column(true_dat)
    
    true_dat['switch'] = true_dat['action'] != true_dat['act_1ago']
#     assert len(raw_dat['fullIDs']) * 782 == true_dat.shape[0]
    
    return true_dat

# # Example use
# true_dat_all = format_mouse_data(raw_dat)
# true_dat_all

In [11]:
# # Summarize over session to count number of trials
# def get_n_trial_dat(true_dat):
    
#     n_trial_dat = true_dat[['animal', 'session', 'action', 'sex', 'agegroup']
#                           ].groupby(['animal', 'session', 'sex', 'agegroup']
#                           ).agg(['count']).reset_index().rename(columns={'action': 'n_trials'})
#     return n_trial_dat

# # # Example use
# # get_n_trial_dat(true_dat_all)

# Functions to load and format simulated data

In [12]:
# Get remaining info for this simulation
def format_sim_data(sim_dat, model_name, selected_mouse=0, save_as_csv=True):
    
    """
    Takes raw `sim_data` (e.g., read in from os.path.join(fitted_param_dir, 'simulations/simulated_mice_{}_nagents{}.csv'.format(model_name, n_agents)))
    Adds all the columns necessary for behavioral analyses.
    """
    
    # Pull out behavior of one specific simulation into separate columns
    for col in ['actionall', 'rewardall', 'correctall']:
        sim_dat[col[:-3]] = sim_dat[col].apply(lambda x: x[1 + 2 * selected_mouse]).astype('int')
        
    # Add gender and session for each simulated animal
    for animal in np.unique(sim_dat.animal):
        print('animal {} / {}'.format(animal, len(np.unique(sim_dat.animal))))
        sub_idx_a = sim_dat.animal == animal
        sim_dat.loc[sub_idx_a, 'gender'] = get_info_from_fullID(sim_dat.loc[sub_idx_a, 'fullID'].reset_index(drop=True)[0], sum(sub_idx_a))['gender']
        sim_age = sim_dat.loc[sub_idx_a, 'age']
        sim_dat.loc[sub_idx_a, 'session'] = sim_age - min(sim_age)

        # Add rew_1ago, rew_2ago, switch for each session
        for age in np.unique(sim_dat.loc[sub_idx_a, 'age']):
            sub_idx_aa = (sim_dat.animal == animal) * (sim_dat.age == age)
            sim_dat.loc[sub_idx_aa, 'rew_1ago'] = np.append(np.nan, sim_dat.loc[sub_idx_aa, 'reward'][:-1])
            sim_dat.loc[sub_idx_aa, 'rew_2ago'] = np.append(np.nan, sim_dat.loc[sub_idx_aa, 'rew_1ago'][:-1])
            sim_dat.loc[sub_idx_aa, 'switch'] = np.append(np.nan, np.abs(np.diff(sim_dat.loc[sub_idx_aa, 'action'].astype('int'))))

            # Add block_length, block_trial, trials2swtich for each block
            for block in np.unique(sim_dat.loc[sub_idx_aa, 'block']):
                sub_idx_baa = (sim_dat.block == block) * (sim_dat.animal == animal) * (sim_dat.age == age)
                block_length = sum(sub_idx_baa)

                if block_length > 5:
                    block_trial = range(block_length)
                    trials2switch = np.append(np.arange(block_length-3), np.arange(-3, 0))
                else:
                    block_trial = [np.nan] * block_length
                    trials2switch = [np.nan] * block_length

                sim_dat.loc[sub_idx_baa, 'block_length'] = block_length
                sim_dat.loc[sub_idx_baa, 'block_trial'] = block_trial
                sim_dat.loc[sub_idx_baa, 'trials2switch'] = trials2switch

    # Format gender; add rew_12ago and agegroup
#     true_dat['gender'] = true_dat['gender'].replace({1: 'Male', 2: 'Female'})
    sim_dat['rew_12ago'] = [str(r1)[0] + str(r2)[0] for r1, r2 in zip(sim_dat['rew_1ago'], sim_dat['rew_2ago'])]
    sim_dat.loc[sim_dat.age > 65, 'agegroup'] = 'Adult'  # adults: 69-90
    sim_dat.loc[sim_dat.age < 65, 'agegroup'] = 'Juvenile'  # juveniles: 38-60

    # Save as csv
    if save_as_csv:
        sim_dat.to_csv(os.path.join(mouse_data_dir, 'formatted_sim_mouse_data_{}.csv'.format(model_name)), index=False)
        
    return sim_dat

# # Example use
# n_agents = 10
# modelnames = [f for f in os.listdir(fitted_param_dir) if ('.csv' in f) and ('params' in f) and ('B' not in f)]
# model_names = [modelname.split('_')[1] for modelname in modelnames]
# simulation_dir = os.path.join(fitted_param_dir, 'simulations/simulated_mice_{}_nagents{}.csv'.format(model_name, n_agents))
# sim_dat = pd.read_csv(simulation_dir)
# format_sim_data(sim_dat, model_name, selected_mouse=0, save_as_csv=True)

# Read in and clean mouse data

In [13]:
# Read in and format mouse data
true_dat_all_dir = os.path.join(mouse_data_dir, 'mouse_true_dat_raw.csv')
print("Reading in raw mouse data...")
raw_dat = load_mouse_data(mouse_data_dir)
print("Cleaning mouse data...")
true_dat_all = format_mouse_data(raw_dat)
print("Save cleaned csv.                            ")
true_dat_all.to_csv(true_dat_all_dir, index=False)
true_dat_all

Reading in raw mouse data...
Cleaning mouse data...
Save cleaned csv.                            


,action,reward,rt,correct,trial,block,fullID,agegroup,sex,age,...,act_18ago,act_19ago,block_length,block_trial,trials2switch,session,rew_1&2ago,cor_1&2ago,act_1&2ago,meta
0,0.0,0.0,1.15620,0.0,0,0.0,1404012,Juvenile,Male,40,...,NaN,NaN,12.0,0.0,0.0,0.0,nn,nn,nn,1-3
1,0.0,0.0,0.28317,0.0,1,0.0,1404012,Juvenile,Male,40,...,NaN,NaN,12.0,1.0,1.0,0.0,0n,0n,0n,1-3
2,1.0,1.0,0.46666,1.0,2,0.0,1404012,Juvenile,Male,40,...,NaN,NaN,12.0,2.0,2.0,0.0,00,00,00,1-3
3,1.0,1.0,1.28200,1.0,3,0.0,1404012,Juvenile,Male,40,...,NaN,NaN,12.0,3.0,3.0,0.0,10,10,10,1-3
4,1.0,1.0,0.96366,1.0,4,0.0,1404012,Juvenile,Male,40,...,NaN,NaN,12.0,4.0,4.0,0.0,11,11,11,1-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,NaN,NaN,NaN,NaN,768,NaN,1309021,Adult,Female,90,...,NaN,NaN,NaN,NaN,NaN,15.0,nn,nn,nn,nan
769,NaN,NaN,NaN,NaN,769,NaN,1309021,Adult,Female,90,...,NaN,NaN,NaN,NaN,NaN,15.0,nn,nn,nn,nan
770,NaN,NaN,NaN,NaN,770,NaN,1309021,Adult,Female,90,...,NaN,NaN,NaN,NaN,NaN,15.0,nn,nn,nn,nan
771,NaN,NaN,NaN,NaN,771,NaN,1309021,Adult,Female,90,...,NaN,NaN,NaN,NaN,NaN,15.0,nn,nn,nn,nan


In [14]:
# Subset 500 trials and 10 sessions
true_dat = true_dat_all.loc[
    (true_dat_all.trial < n_trial_cutoff) &  # cap at 500 trials
    (true_dat_all.session <= n_session_cutoff)  # cap at 10 sessions
]
excl_dat = true_dat.loc[np.isnan(true_dat.action)].groupby(['fullID']).aggregate('mean').reset_index()[['fullID']]  # , 'age', 'animal', 'session'

true_dat['excluded'] = False
for fullID in excl_dat.fullID:
    true_dat.loc[true_dat.fullID == fullID, 'excluded'] = True
    
true_dat = true_dat.loc[np.invert(true_dat.excluded)]
assert np.mean(true_dat.excluded) == 0
true_dat

<ipython-input-14-9399128ab1ff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\maria\Anaconda3\envs\notebooks\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\maria\Anaconda3\envs\notebooks\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,action,reward,rt,correct,trial,block,fullID,agegroup,sex,age,...,act_19ago,block_length,block_trial,trials2switch,session,rew_1&2ago,cor_1&2ago,act_1&2ago,meta,excluded
0,0.0,0.0,1.15620,0.0,0,0.0,1404012,Juvenile,Male,40,...,NaN,12.0,0.0,0.0,0.0,nn,nn,nn,1-3,False
1,0.0,0.0,0.28317,0.0,1,0.0,1404012,Juvenile,Male,40,...,NaN,12.0,1.0,1.0,0.0,0n,0n,0n,1-3,False
2,1.0,1.0,0.46666,1.0,2,0.0,1404012,Juvenile,Male,40,...,NaN,12.0,2.0,2.0,0.0,00,00,00,1-3,False
3,1.0,1.0,1.28200,1.0,3,0.0,1404012,Juvenile,Male,40,...,NaN,12.0,3.0,3.0,0.0,10,10,10,1-3,False
4,1.0,1.0,0.96366,1.0,4,0.0,1404012,Juvenile,Male,40,...,NaN,12.0,4.0,4.0,0.0,11,11,11,1-3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,3.70600,0.0,495,26.0,1308321,Adult,Female,83,...,0.0,25.0,2.0,2.0,10.0,00,10,10,9-11,False
496,0.0,0.0,1.91100,0.0,496,26.0,1308321,Adult,Female,83,...,0.0,25.0,3.0,3.0,10.0,00,01,01,9-11,False
497,1.0,1.0,3.54000,1.0,497,26.0,1308321,Adult,Female,83,...,1.0,25.0,4.0,4.0,10.0,00,00,00,9-11,False
498,1.0,0.0,0.52400,1.0,498,26.0,1308321,Adult,Female,83,...,0.0,25.0,5.0,5.0,10.0,10,10,10,9-11,False


In [15]:
# true_dat.loc[true_dat.rt < 0].groupby(['fullID']).aggregate('max').reset_index()[['age', 'sex', 'animal', 'fullID', 'block', 'rt']]

In [16]:
# # Remove RTs < 0
# true_dat.loc[true_dat.rt < 0, 'rt'] = np.nan

In [17]:
# RT distribution
np.quantile(true_dat.loc[np.invert(np.isnan(true_dat.rt)), 'rt'], [0.8, 0.9, 0.95, 0.98, 0.99])

array([0.7835  , 1.17    , 1.717525, 2.583   , 3.134   ])

In [18]:
# # Remove RTs > 10 sec
# true_dat.loc[true_dat.rt > rt_cut_off, 'rt'] = np.nan
# true_dat

In [19]:
true_dat.describe()

,action,reward,rt,correct,trial,block,fullID,age,animal,rew_1ago,...,act_14ago,act_15ago,act_16ago,act_17ago,act_18ago,act_19ago,block_length,block_trial,trials2switch,session
count,191500.000000,191500.000000,191500.000000,191500.000000,191500.000000,191500.000000,1.915000e+05,191500.000000,191500.000000,191117.000000,...,186138.000000,185755.000000,185372.000000,184989.000000,184606.000000,184223.000000,191500.000000,191500.000000,191500.000000,191500.000000
mean,0.512308,0.603091,0.652097,0.797869,249.500000,9.850115,1.934056e+06,55.814621,19.284595,0.603060,...,0.512781,0.512788,0.512758,0.512712,0.512621,0.512650,26.756642,12.746658,9.818480,4.804178
std,0.499850,0.489258,0.531991,0.401590,144.337655,6.159996,1.103933e+06,16.969518,11.052713,0.489265,...,0.499838,0.499838,0.499839,0.499840,0.499842,0.499841,8.293347,9.047259,9.028474,3.066850
min,0.000000,0.000000,0.004000,0.000000,0.000000,0.000000,1.075110e+05,38.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,-3.000000,0.000000
25%,0.000000,0.000000,0.391000,1.000000,124.750000,4.000000,1.007521e+06,42.000000,10.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,5.000000,2.000000,2.000000
50%,1.000000,1.000000,0.487830,1.000000,249.500000,10.000000,1.904722e+06,46.000000,19.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,27.000000,11.000000,8.000000,5.000000
75%,1.000000,1.000000,0.693500,1.000000,374.250000,15.000000,2.904222e+06,76.000000,29.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,32.000000,19.000000,16.000000,7.000000
max,1.000000,1.000000,4.000000,1.000000,499.000000,26.000000,3.806012e+06,86.000000,38.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,62.000000,61.000000,58.000000,10.000000


In [20]:
assert np.sum(np.sum(np.isnan(true_dat[['action', 'reward', 'rt', 'correct']]))) == 0

## Save true_dat csv and use this for modeling!

In [21]:
true_dat.to_csv(os.path.join(mouse_data_dir, 'true_dat.csv'), index=False)